In [10]:
run src/midi_processing.py

In [11]:
oh = one_hot_song(corpus[67])
song = converter.parse(corpus[67])
this_oh = one_hot_song('data/leads_with_rests/Kygo_feat_Whitney_Houston_HigherLove.mid')
events = np.nonzero(this_oh)[1]

In [12]:
def one_hot_to_midi(one_hot_song, inst='Saxophone'):
    #events = np.nonzero(one_hot_encoded_song)[0]
    events = np.nonzero(one_hot_song)[1]
    song_iter = enumerate(events) 
    this_stream = stream.Stream()
    part = stream.Part()
    inst = instrument.Instrument(inst)
    temp = tempo.MetronomeMark('animato')
    met = meter.TimeSignature('4/4')
    this_stream.append(part)
    part.append(inst)
    part.append(temp)
    part.append(met)
    for idx, val in song_iter:
        if idx == 0 and 256 <= val <= 355:
            time, count = get_rest_duration(one_hot_song, idx)
            dur = temp.secondsToDuration(time)
            r = note.Rest(duration=dur)
            part.append(r)
            [next(song_iter) for _ in range(count-1)]
        elif check_for_chord(one_hot_song, idx):
            chord_length = get_note_count_chord(one_hot_song,idx)
            vel_index = idx + chord_length
            ts_index = vel_index + 1
            time, count = get_rest_duration(one_hot_song, ts_index)
            if time <= 0.0:
                time = .04
            dur = temp.secondsToDuration(time)
            vel = (events[vel_index] - 355) * 4 - 1
            chord_notes = get_chord(one_hot_song, idx)
            ch = chord.Chord(duration=dur)
            for num in chord_notes:
                p = pitch.Pitch(num)
                string = p.nameWithOctave
                ch.add(string)                
            ch.volume = volume.Volume(velocity=vel)
            part.append(ch)
            step = chord_length + count +2 #add two for duration and note off events
            [next(song_iter) for _ in range(step)]
        elif 256 <= val <= 387 and is_rest(one_hot_song, idx) and idx != 0:
            time, count = get_rest_duration(one_hot_song, idx)
            dur = temp.secondsToDuration(time)
            r = note.Rest(duration=dur)
            part.append(r)
            [next(song_iter) for _ in range(count-1)]
        else:
            time, count = get_rest_duration(one_hot_song, idx+2)
            vel = (events[idx+1] - 355) * 4 - 1
            dur = temp.secondsToDuration(time)
            n = note.Note(val,duration=dur)
            n.volume.velocity = vel
            part.append(n)
            [next(song_iter) for _ in range(count + 2)] #add one for note off event
    return this_stream

In [13]:
fuck_you = one_hot_to_midi(this_oh)

In [14]:
mf = midi.translate.streamToMidiFile(fuck_you)
mf.open('data/rencoded/test.mid', 'wb')
mf.write()
mf.close()